# Celestial coordinates

The [astropy.coordinates](https://docs.astropy.org/en/stable/coordinates/index.html) sub-package in astropy provides a way to represent and manipulate celestial coordinates. It supports many different coordinate systems and transformations, and includes support for velocities in addition to positions.


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Create SkyCoord objects</li>
<li>Access individual coordinates</li>
<li>Format coordinates</li>
<li>Transforming coordinates</li>
<li>Using 3D coordinates and velocities</li>
<li>Calculating separations</li>
<li>Using arrays of coordinates</li>
</ul>

</div>

</section>


## Documentation

This notebook only shows a subset of the functionality in astropy.coordinates. For more information about the features presented below as well as other available features, you can read the
[astropy.coordinates documentation](https://docs.astropy.org/en/stable/coordinates/).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('image', origin='lower')
plt.rc('figure', figsize=(10, 6))

## Creating SkyCoord objects

The main user-facing class in astropy.coordinates is the ``SkyCoord`` class:

In [ ]:
from astropy.coordinates import SkyCoord

A SkyCoord object can represent one or more coordinates, and there are a number of different ways to initialize such an object. If you have the coordinates as quantities, you can do:

In [ ]:
from astropy import units as u
coord1 = SkyCoord(13.443 * u.deg, -33.298 * u.deg)
coord1

As you can see, the default is to assume that the coordinates are in the ICRS frame and represent Right Ascension and Declination. You can also initialize coordinates from strings:

In [ ]:
coord2 = SkyCoord('13:18:22', '-21:43:34', unit=(u.hourangle, u.deg))
coord2

In [ ]:
coord3 = SkyCoord('13h18m22s -21d43m34s')
coord3

To create coordinates in other frames - for example Galactic coordinates - you can use the ``frame=`` argument:

In [ ]:
coord4 = SkyCoord(l=13.4 * u.deg, b=0.1 * u.deg, frame='galactic')
coord4

Note that here we've also specified the names of the coordinates explicitly using ``l=`` and ``b=`` (they are not required but can make code more readable).

## Initialization from SIMBAD identifier

If you are doing quick calculations and want to use the coordinates of a source in the SIMBAD database, you can easily create a ``SkyCoord`` object from the object name:

In [ ]:
m42 = SkyCoord.from_name('M42')
m42

However, note that this should not be used when precision is critical - the positions are approximate especially for extended sources.

## Accessing individual coordinates

Once you have a coordinate object, you can access the individual coordinates using attributes:

In [ ]:
coord3.ra

In [ ]:
type(coord3.ra)

The Longitude class is a sub-class of Quantity so you can convert it to specific units as you would a quantity:

In [ ]:
coord3.ra.to(u.hourangle).value

but the Longitude class also provides shortcuts for this:

In [ ]:
coord3.ra.hourangle

## Formatting coordinates

You can format a coordinate object as a string using ``to_string()``:

In [ ]:
coord3.to_string()

The default format uses floating point values, but you may want instead to use standard sexagesimal format: 

In [ ]:
coord2.to_string('hmsdms')

You can control the precision using the ``precision=`` keyword argument:

In [ ]:
coord2.to_string('hmsdms', precision=5)

## Transforming coordinates

``SkyCoord`` objects can be transformed between frames using the ``.transform_to()`` method:

In [ ]:
coord1.transform_to('galactic')

You can also specify frame objects if you need more control:

In [ ]:
from astropy.coordinates import FK5
coord1.transform_to(FK5(equinox='J2020'))

A more advanced example is to convert the coordinates to altitude/azimuth as seen from a particular observer at a specific time (in UTC):

In [ ]:
from astropy.coordinates import EarthLocation, AltAz
from astropy.time import Time

In [ ]:
madrid = EarthLocation(lat=40.4168 * u.deg, lon=-3.7038*u.deg, height=667 * u.m)
utc_time = Time.now()

In [ ]:
coord_altaz = coord1.transform_to(AltAz(obstime=utc_time, location=madrid))

In [ ]:
coord_altaz.az

In [ ]:
coord_altaz.alt

## 3D coordinates

The above examples are for coordinates on the sky, which are assumed to be at infinity. However, it is also possible to specify a third coordinate (the distance to the object):

In [ ]:
crab = SkyCoord('05h34m 22d00m', distance=2 * u.kpc)
crab

We can convert this to the Galactocentric frame, which is a frame that represents coordinates in cartesian coordinates by default (centered on the center of the Milky Way).

In [ ]:
crab_galcen = crab.transform_to('galactocentric')
crab_galcen.x

In [ ]:
crab_galcen.y

In [ ]:
crab_galcen.z

## Velocities/proper motions

In addition to being able to represent fixed positions in space, it is also possible to add velocities to coordinates, for example to express radial velocity, proper motions, or 3D space motion. For example, to add a radial velocity and proper motion to a coordinate in the ICRS frame, you can do:

In [ ]:
coord4 = SkyCoord(ra=13.443 * u.deg,
                  dec=-33.298 * u.deg, distance=1 * u.kpc,
                  pm_ra_cosdec=0.1 * u.arcsec / u.yr,
                  pm_dec=-0.08 * u.arcsec / u.yr,
                  radial_velocity=20 * u.km / u.s)
coord4

Velocities are then correctly propagated when transforming to different systems:

In [ ]:
coord4.transform_to('galactic')

In [ ]:
coord4.transform_to('galactocentric')

## Separations

Given two coordinates, we can determine the separation between them on the sky:

In [ ]:
coord1.separation(coord3)

We can also do this in 3D to find a physical distance:

In [ ]:
crab_galcen.separation_3d(coord4)

## Using arrays in coordinates

A common use case is the need to represent large catalogs of coordinates - in this case, it is very inefficient to use a ``SkyCoord`` object for each source. Instead, you can store arrays of coordinates inside ``SkyCoord``:

In [ ]:
coord5 = SkyCoord(ra=[1, 2, 3] * u.deg, dec=[4, 5, 6] * u.deg)
coord5

Most of the operations shown above should then also work seamlessly with arrays:

In [ ]:
coord5.separation(coord1).to(u.deg)

## Solar system bodies

A [get_body](https://docs.astropy.org/en/stable/api/astropy.coordinates.get_body.html) convenience function is provided in astropy.coordinates to retrieve the 3D coordinates of solar system bodies. This can be done either using default built-in ephemeris, or the more accurate JPL ephemeris (see the ``get_body`` documentation). For now, we will use the default:

In [ ]:
from astropy.time import Time
from astropy.coordinates import get_body
mars = get_body('mars', Time.now())

In [ ]:
mars


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>The answer to some of the following can be found in <a href="https://docs.astropy.org/en/stable/time/index.html">the documentation</a>!</p>
<ol>
<li>Find the coordinates of the Crab Nebula (M1) in ICRS coordinates, and convert them to Galactic Coordinates.</li>
<li>Find the separation on the sky between the Crab Nebula and the star 'T Tauri' in degrees</li>
<li>Generate random positions in the Galactic plane (between l=0 and l=360 and b=-1 and b=+1), then make a plot showing the position on the sky of these points in FK5.</li>
<li>Make a plot showing the altitude above the horizon of the Sun as seen from Madrid over the course of today</li>
</ol>

</div>

</section>


In [ ]:
#1
crab = SkyCoord.from_name('M1')
print(crab.transform_to('galactic'))

In [ ]:
#2
ttauri = SkyCoord.from_name('T Tauri')
print(ttauri.separation(crab).to(u.deg).value)

In [ ]:
#3
import numpy as np
l = np.random.uniform(0, 360, 1000) * u.deg
b = np.random.uniform(-1, 1, 1000) * u.deg
coord = SkyCoord(l, b, frame='galactic').transform_to('fk5')
plt.plot(coord.ra.hourangle, coord.dec.deg, 'o')
plt.xlabel('Right Ascension')
plt.ylabel('Declination')

In [ ]:
#4
start = Time('2019-10-23T00:00:00')
relative = np.linspace(0, 24, 1000) * u.hour
times = start + relative
sun = get_body('sun', times)
sun_altaz = sun.transform_to(AltAz(obstime=times, location=madrid))
plt.plot(relative, sun_altaz.alt)
plt.axhline(0)
plt.xlabel('Time (UTC)')
plt.ylabel('Sun Altitude')

<center><i>This notebook was written by <a href="https://aperiosoftware.com/">Aperio Software Ltd.</a> &copy; 2019, and is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License (CC BY 4.0)</a></i></center>

![cc](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by.svg)